In [1]:
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun

In [2]:
arxiv_api = ArxivAPIWrapper()
arxiv=ArxivQueryRun(api_wrapper=arxiv_api)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000))

In [21]:
###arxiv.invoke('')

In [9]:
wikipedia_api=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=400)
wiki=WikipediaQueryRun(api_wrapper=wikipedia_api)
wiki.name


'wikipedia'

In [10]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"]="ReAct-agent"


In [5]:
### Custom Functions for tools
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools=[arxiv,wiki,add,multiply,divide]

In [11]:
### Tavily Search Tool
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()



In [12]:
tavily.invoke('Provide me the recent AI news for 2025')

c:\Users\Prema\agent\env\Lib\site-packages\langsmith\client.py:500: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6575-e856-7861-91ea-aa6f8a1f9911,id=019b6575-e856-7861-91ea-aa6f8a1f9911


[{'title': '60 of our biggest AI announcements in 2025',
  'url': 'https://blog.google/technology/ai/google-ai-news-recap-2025/',
  'content': 'We didn’t let up as we rounded the bend of 2025. Instead, we squeezed even more AI news into an already jam-packed year with the introduction of Gemini 3 Flash, our annual Year in Search release and still more announcements to round us out. Here were some of the top Google AI news stories of the month: [...] # 60 of our biggest AI announcements in 2025\n\nDec 22, 2025\n\nCatch up on major Google AI moments this year across Gemini, Search, Pixel and more.\n\nzahra_headshot\n\n## General summary\n\nGoogle had a busy 2025 with many AI product and feature announcements. Gemini updates, AI Mode in Search and new hardware were just a few of the highlights. Look forward to more AI advancements from Google in 2026. [...] As 2025 comes to a close, it’s only natural to look back on our biggest AI moments of the year. And what a year it’s been — we shared

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6575-e856-7861-91ea-aa6f8a1f9911,id=019b6575-e856-7861-91ea-aa6f8a1f9911


In [13]:
### Combine all the tools in the list

tools=[arxiv,wiki,tavily,add,divide,multiply]

In [19]:
## Initialize my LLM model

from langchain_groq import ChatGroq

llm=ChatGroq(model="llama-3.1-8b-instant")

llm_with_tools=llm.bind_tools(tools)



In [ ]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage
llm_with_tools.invoke([HumanMessage(content=f"What is the recent AI News")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'bsb8w0cwp', 'function': {'arguments': '{"query":"Recent AI News"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 744, 'total_tokens': 764, 'completion_time': 0.039941864, 'completion_tokens_details': None, 'prompt_time': 0.082619692, 'prompt_tokens_details': None, 'queue_time': 0.051881287, 'total_time': 0.122561556}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b657a-dd00-7083-8588-e74ab75f3f77-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Recent AI News'}, 'id': 'bsb8w0cwp', 'type': 'tool_call'}], usage_metadata={'input_tokens': 744, 'output_tokens': 20, 'total_tokens': 764})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b657a-dd00-7083-8588-e74ab75f3f77,id=019b657a-dd00-7083-8588-e74ab75f3f77
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b657a-dd00-7083-8588-e74ab75f3f77,id=019b657a-dd00-7083-8588-e74ab75f3f77


In [22]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage
## State Schema
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages
class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [23]:
### Entire Chatbot With LangGraph
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

### Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools","tool_calling_llm")


graph = builder.compile()


In [26]:
messages=graph.invoke({"messages":HumanMessage(content="Provide me the top 10 recent AI news for December 28th 2025,add 5 plus 5 and then multiply by 10")})
for m in messages['messages']:
    m.pretty_print()

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-18b8-7db3-b1a1-8c9dea7343ce; trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-18c1-7920-9c50-1bcf5cd836c3; trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-18c3-73f2-bccb-662378092bb4
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-18c3-73f2-bccb-662378092bb4; trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-1b24-7872-8789-ed95c761111b; trace=019b6591-

================================ Human Message =================================

Provide me the top 10 recent AI news for December 28th 2025,add 5 plus 5 and then multiply by 10
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (2ktsrghjg)
 Call ID: 2ktsrghjg
  Args:
    query: AI news December 28th 2025
  add (hq8h3ywx9)
 Call ID: hq8h3ywx9
  Args:
    a: 5
    b: 5
  multiply (kjhkgjv7q)
 Call ID: kjhkgjv7q
  Args:
    a: 10
    b: 10
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Big Tech's AI money loop: Boom, bubble, or bottleneck? - eMarketer", "url": "https://www.emarketer.com/content/big-tech-s-ai-money-loop--boom--bubble--bottleneck", "content": "Learn More →\n\nNewsroom\n\nSee our latest press releases, news articles or download our press kit.\n\nLearn More →\n\nContact Us\n\nSpeak to a member of our team to learn more about 

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-2ba8-7d40-9a28-64cf1870eb8a; trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-2f11-7083-a389-ca682629ff32; trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-2f11-7083-a389-ca682629ff32; trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-2ba7-76a3-867e-d360117e8e3e; trace=019b6591-18b8-7db3-b1a1-8c9dea7343ce,id=019b6591-18b8-7db3-b1a1-8c9dea7343ce


In [ ]:
messages=graph.invoke({"messages":HumanMessage(content="What is machine learning")})
for m in messages['messages']:
    m.pretty_print()